In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ratings_df = pd.read_csv('ratingsProcessed.csv')
train_rating_df = pd.read_csv('train_rating.csv')
test_rating_df = pd.read_csv('test_rating.csv')
userNo,movieNo = ratings_df['userId'].max()+1,ratings_df['movieId'].max()+1
testNo = np.shape(test_rating_df)[0]
train_global_rating_mean = np.mean(train_rating_df["rating"])
train_rating = np.zeros((userNo,movieNo))
for index ,row in train_rating_df.iterrows():
    train_rating[int(row['userId']),int(row['movieId'])] = row['rating']
test_rating = np.zeros((userNo,movieNo))
for index ,row in test_rating_df.iterrows():
    test_rating[int(row['userId']),int(row['movieId'])] = row['rating']
train_record = train_rating>0
train_record = np.array(train_record,dtype=int)
test_record = test_rating>0
test_record = np.array(test_record,dtype=int)

In [3]:
def createLFM(f):
    LFM_P = tf.Variable(tf.random_normal([userNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    LFM_Q = tf.Variable(tf.random_normal([movieNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    LFM_predict_rating = tf.matmul(LFM_P,LFM_Q,transpose_b=True)
    LFM_loss = tf.reduce_sum(((train_rating-LFM_predict_rating)*train_record)**2) + lamda*((tf.reduce_sum(LFM_P)**2)+(tf.reduce_sum(LFM_Q)**2))
    return LFM_loss,LFM_predict_rating

In [4]:
def createBSVD(f):
    BIAS_u = train_global_rating_mean
    BIAS_bu = tf.Variable(tf.zeros((userNo,1),dtype=tf.float32))
    BIAS_bi = tf.Variable(tf.zeros((1,movieNo),dtype=tf.float32))
    BIAS_P = tf.Variable(tf.random_normal([userNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    BIAS_Q = tf.Variable(tf.random_normal([movieNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    BIAS_predict_rating = BIAS_u + BIAS_bu + BIAS_bi + tf.matmul(BIAS_P,BIAS_Q,transpose_b=True)
    BIAS_loss = tf.reduce_sum(((train_rating-BIAS_predict_rating)*train_record)**2) + lamda*((tf.reduce_sum(BIAS_P)**2) + (tf.reduce_sum(BIAS_Q)**2) + (tf.reduce_sum(BIAS_bu)**2) + (tf.reduce_sum(BIAS_bi)**2))
    return BIAS_loss,BIAS_predict_rating

In [5]:
def createSVD(f):
    SVD_u = train_global_rating_mean
    SVD_bu = tf.Variable(tf.zeros((userNo,1),dtype=tf.float32))
    SVD_bi = tf.Variable(tf.zeros((1,movieNo),dtype=tf.float32))
    SVD_P = tf.Variable(tf.random_normal([userNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    SVD_Q = tf.Variable(tf.random_normal([movieNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    SVD_y = tf.Variable(tf.random_normal([movieNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    SVD_Nu = np.sum(train_record,axis=1)
    N = np.zeros((userNo,movieNo),dtype=np.float32)
    for i in range(0,userNo):
        if SVD_Nu[i] == 0:
            SVD_Nu[i] = 1
    for i in range(0,movieNo):
        N[:,i] = SVD_Nu
    SVD_Nu = N**0.5
    SVD_Qy=tf.matmul(SVD_Q,SVD_y,transpose_b=True)
    SVD_tr = tf.constant(train_record,dtype=tf.float32)
    SVD_predict_rating = SVD_u + SVD_bu + SVD_bi + tf.matmul(SVD_P,SVD_Q,transpose_b=True) + tf.matmul(SVD_tr,SVD_Qy)/SVD_Nu
    SVD_loss = tf.reduce_sum(((train_rating-SVD_predict_rating)*train_record)**2) + lamda*((tf.reduce_sum(SVD_P)**2) + (tf.reduce_sum(SVD_Q)**2) + (tf.reduce_sum(SVD_bu)**2) + (tf.reduce_sum(SVD_bi)**2) + (tf.reduce_sum(SVD_y)**2))
    return SVD_loss,SVD_predict_rating

In [6]:
lamda = 0.01

In [7]:
learning_rate = 1e-4

In [8]:
filename='tensorboard\\f'

In [9]:
#loss,predict = createLFM(9)
loss,predict = createSVD(9)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)
RMSE = (tf.reduce_sum(((test_rating-predict)*test_record)**2)/testNo)**0.5
tf.summary.scalar("RMSE",RMSE)
summaryMerged = tf.summary.merge_all()
writer = tf.summary.FileWriter(filename)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [10]:
for fc in [10,30,50,80,100,150,200]:
    #loss,predict = createLFM(fc)
    #loss,predict = createBSVD(fc)
    loss,predict = createSVD(fc)
    train = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(500): 
        sess.run(train)
    movie_summary = sess.run(summaryMerged)
    writer.add_summary(movie_summary,fc)
    print(fc)

10
30
50
80
100
150
200
